<a href="https://colab.research.google.com/github/viincci/Jse_index/blob/main/mainBitMEXLiveTrader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q yfinance bitmex matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.2 MB/s eta 0:00:00


In [2]:

# -*- coding: utf-8 -*-
import numpy as np
import bitmex
import time
import logging
import json
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import logging
import yfinance as yf
from datetime import datetime, timedelta

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler("test_bitmex.log")
    ]
)
logger = logging.getLogger(__name__)

In [3]:
class BitMEXTestAPI:
    def __init__(self, api_key, api_secret, test=True, symbol='XBTUSD'):
        """Initialize with BitMEX API credentials"""
        self.client = bitmex.bitmex(
            test=test,
            api_key=api_key,
            api_secret=api_secret
        )
        self.symbol = symbol
        logger.info(f"BitMEXTestAPI initialized for {symbol} {'on testnet' if test else 'on mainnet'}")
        print(f"BitMEXTestAPI initialized for {symbol} {'on testnet' if test else 'on mainnet'}")

    def get_profile_info(self):
        """Get account profile information"""
        try:
            # Get user information
            user_info = self.client.User.User_get().result()[0]

            # Get account balance
            margin = self.client.User.User_getMargin().result()[0]

            # Get open positions
            positions = self.client.Position.Position_get(
                filter=json.dumps({"symbol": self.symbol})
            ).result()[0]

            # Format and return information
            profile_info = {
                "user": {
                    "id": user_info.get('id'),
                    "username": user_info.get('username'),
                    "email": user_info.get('email'),
                    "account": user_info.get('account')
                },
                "balance": {
                    "wallet_balance": margin.get('walletBalance'),
                    "margin_balance": margin.get('marginBalance'),
                    "available_margin": margin.get('availableMargin'),
                    "unrealized_pnl": margin.get('unrealisedPnl'),
                    "realized_pnl": margin.get('realisedPnl')
                },
                "positions": [{
                    "symbol": pos.get('symbol'),
                    "current_qty": pos.get('currentQty'),
                    "avg_entry_price": pos.get('avgEntryPrice'),
                    "leverage": pos.get('leverage'),
                    "liquidation_price": pos.get('liquidationPrice'),
                    "unrealized_pnl": pos.get('unrealisedPnl'),
                    "realized_pnl": pos.get('realisedPnl')
                } for pos in positions] if positions else []
            }

            # Log the profile information
            logger.info("Profile information retrieved successfully")
            print("Profile information retrieved successfully")

            logger.info(f"Account: {profile_info['user']['username']}")
            print(f"Account: {profile_info['user']['username']}")

            logger.info(f"Wallet Balance: {profile_info['balance']['wallet_balance'] / 100000000:.8f} XBT")
            print(f"Wallet Balance: {profile_info['balance']['wallet_balance'] / 100000000:.8f} XBT")

            logger.info(f"Available Margin: {profile_info['balance']['available_margin'] / 100000000:.8f} XBT")
            print(f"Available Margin: {profile_info['balance']['available_margin'] / 100000000:.8f} XBT")

            if profile_info['positions']:
                for pos in profile_info['positions']:
                    logger.info(f"Position: {pos['symbol']} | Qty: {pos['current_qty']} | Entry: {pos['avg_entry_price']}")
                    print(f"Position: {pos['symbol']} | Qty: {pos['current_qty']} | Entry: {pos['avg_entry_price']}")
            else:
                logger.info("No open positions")
                print("No open positions")

            return profile_info

        except Exception as e:
            logger.error(f"Error getting profile information: {str(e)}")
            print(f"Error getting profile information: {str(e)}")
            return None

    def open_test_position(self, side="Buy", quantity=100, order_type="Market"):
        """Open a test position"""
        try:
            logger.info(f"Opening test {side} position for {quantity} contracts")

            # Execute the order
            order = self.client.Order.Order_new(
                symbol=self.symbol,
                side=side,
                orderQty=quantity,
                ordType=order_type
            ).result()[0]

            # Log order details
            logger.info(f"Order placed: {order['ordStatus']} | OrderID: {order['orderID']}")
            print(f"Order placed: {order['ordStatus']} | OrderID: {order['orderID']}")

            logger.info(f"Order details: {side} {quantity} contracts at {order.get('price', 'market price')}")
            print(f"Order details: {side} {quantity} contracts at {order.get('price', 'market price')}")

            # Get updated position information
            time.sleep(2)  # Wait for order to settle
            self.get_profile_info()

            return order

        except Exception as e:
            logger.error(f"Error opening test position: {str(e)}")
            print(f"Error opening test position: {str(e)}")
            return None

    def close_all_positions(self):
        """Close all open positions"""
        try:
            # Get current positions
            positions = self.client.Position.Position_get(
                filter=json.dumps({"symbol": self.symbol})
            ).result()[0]

            if not positions:
                logger.info("No positions to close")
                print("No positions to close")
                return None

            # For each position, create a closing order
            for position in positions:
                symbol = position['symbol']
                current_qty = position['currentQty']

                if current_qty == 0:
                    logger.info(f"No open position for {symbol}")
                    print(f"No open position for {symbol}")
                    continue

                # Determine side for closing (opposite of current position)
                side = "Sell" if current_qty > 0 else "Buy"
                # Order quantity should be the absolute value of current_qty
                qty = abs(current_qty)

                logger.info(f"Closing position: {symbol} | {current_qty} contracts | Side: {side} | Qty: {qty}")
                print(f"Closing position: {symbol} | {current_qty} contracts | Side: {side} | Qty: {qty}")

                # Place closing order
                order = self.client.Order.Order_new(
                    symbol=symbol,
                    side=side,
                    orderQty=qty,
                    ordType="Market"
                ).result()[0]

                logger.info(f"Position closed: {order['ordStatus']} | OrderID: {order['orderID']}")
                print(f"Position closed: {order['ordStatus']} | OrderID: {order['orderID']}")

            # Get updated profile information
            time.sleep(2)  # Wait for orders to settle
            self.get_profile_info()

            return True

        except Exception as e:
            logger.error(f"Error closing positions: {str(e)}")
            print(f"Error closing positions: {str(e)}")
            return None

    def run_test_sequence(self):
        """Run a complete test sequence: check profile, open position, wait, close position"""
        try:
            logger.info("Starting test sequence")
            print("Starting test sequence")

            # 1. Get initial profile information
            logger.info("=== INITIAL PROFILE ===")
            print("=== INITIAL PROFILE ===")
            self.get_profile_info()

            # 2. Open a long position
            logger.info("=== OPENING LONG POSITION ===")
            print("=== OPENING LONG POSITION ===")
            self.open_test_position(side="Buy", quantity=100)

            # 3. Wait a bit
            wait_time = 10
            logger.info(f"Waiting for {wait_time} seconds...")
            print(f"Waiting for {wait_time} seconds...")
            time.sleep(wait_time)

            # 4. Get updated profile
            logger.info("=== PROFILE AFTER OPENING POSITION ===")
            print("=== PROFILE AFTER OPENING POSITION ===")
            self.get_profile_info()

            # 5. Open a short position (this will close the long and go short)
            logger.info("=== OPENING SHORT POSITION ===")
            print("=== OPENING SHORT POSITION ===")
            self.open_test_position(side="Sell", quantity=200)

            # 6. Wait a bit
            logger.info(f"Waiting for {wait_time} seconds...")
            time.sleep(wait_time)

            # 7. Get updated profile
            logger.info("=== PROFILE AFTER REVERSING POSITION ===")
            print("=== PROFILE AFTER REVERSING POSITION ===")
            self.get_profile_info()

            # 8. Close all positions
            logger.info("=== CLOSING ALL POSITIONS ===")
            print("=== CLOSING ALL POSITIONS ===")
            self.close_all_positions()

            # 9. Final profile check
            logger.info("=== FINAL PROFILE ===")
            print("=== FINAL PROFILE ===")
            final_profile = self.get_profile_info()

            logger.info("Test sequence completed successfully")
            print("Test sequence completed successfully")
            return final_profile

        except Exception as e:
            logger.error(f"Error in test sequence: {str(e)}")
            print(f"Error in test sequence: {str(e)}")
            return None

In [4]:
def mainBitMEXAPI():
    # BitMEX API credentials (use your test API key and secret)
    API_KEY = ""  # Your test API key
    API_SECRET = ""  # Your test API secret

    try:
        # Initialize test functions
        test = BitMEXTestAPI(
            api_key=API_KEY,
            api_secret=API_SECRET,
            test=True  # Use testnet
        )

        # Run the test sequence
        test.run_test_sequence()

    except KeyboardInterrupt:
        logger.info("Test stopped by user")
        print("Test stopped by user")
    except Exception as e:
        logger.error(f"Unexpected error: {str(e)}")
        print(f"Unexpected error: {str(e)}")

In [5]:

class SMCStrategy:
    def __init__(self, df, initial_balance=10000, risk_per_trade=0.02):
        """
        Initialize SMC Backtester with historical data and configuration

        Parameters:
        df (pandas.DataFrame): OHLC data with columns 'open', 'high', 'low', 'close'
        initial_balance (float): Starting account balance
        risk_per_trade (float): Risk percentage per trade (0.01 = 1%)
        """
        logger.info(f"Initializing SMC Strategy Backtester with {len(df)} candles")
        print(f"Initializing SMC Strategy Backtester with {len(df)} candles")

        # Ensure dataframe has required columns
        required_cols = ['Open', 'High', 'Low', 'Close']

        # Check if it's a multi-index and convert
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        for col in required_cols:
            if col not in df.columns:
                raise ValueError(f"DataFrame missing required column: {col}")
        # Initialize data and account parameters
        self.df = df.copy()
        # Convert column names to lowercase for consistency with original code
        self.df.columns = [col.lower() for col in self.df.columns]
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.risk_per_trade = risk_per_trade

        # Add columns for SMC analysis
        self.df['higher_high'] = False
        self.df['lower_low'] = False
        self.df['bos_up'] = False
        self.df['bos_down'] = False
        self.df['choch_up'] = False
        self.df['choch_down'] = False
        self.df['bullish_fvg'] = False
        self.df['bearish_fvg'] = False

        # Trading variables
        self.in_trade = False
        self.entry_price = 0
        self.stop_loss = 0
        self.take_profit = 0
        self.position_size = 0
        self.trade_type = None  # 'long' or 'short'

        # Performance tracking
        self.trades = []
        self.equity_curve = [initial_balance]

        logger.info(f"Strategy initialized with {initial_balance} account balance and {risk_per_trade*100}% risk per trade")
        print(f"Strategy initialized with {initial_balance} account balance and {risk_per_trade*100}% risk per trade")

    def identify_structure(self):
        """Identify market structure including highs, lows, BOS and CHoCH"""
        logger.info("Identifying market structure")
        print("Identifying market structure")
        df = self.df

        # Identify Higher Highs and Lower Lows (using a 5-candle lookback)
        window = 5
        for i in range(window, len(df)):
            # Higher High
            if df.iloc[i]['high'] > max(df.iloc[i-window:i]['high']):
                df.loc[df.index[i], 'higher_high'] = True

            # Lower Low
            if df.iloc[i]['low'] < min(df.iloc[i-window:i]['low']):
                df.loc[df.index[i], 'lower_low'] = True

        # Identify Break of Structure (BOS)
        prev_structure_high = df.iloc[0]['high']
        prev_structure_low = df.iloc[0]['low']
        structure_points_high = []
        structure_points_low = []

        for i in range(1, len(df)):
            current_high = df.iloc[i]['high']
            current_low = df.iloc[i]['low']

            # Track significant structure points
            if df.iloc[i]['higher_high']:
                structure_points_high.append((i, current_high))

            if df.iloc[i]['lower_low']:
                structure_points_low.append((i, current_low))

            # BOS Up: Price breaks above recent structure high
            if len(structure_points_high) >= 2:
                last_high_idx, last_high = structure_points_high[-1]
                prev_high_idx, prev_high = structure_points_high[-2]

                if current_low > prev_high and i > last_high_idx + 1:
                    df.loc[df.index[i], 'bos_up'] = True
                    logger.info(f"Bullish BOS detected at index {i}, price: {current_low}")
                    print(f"Bullish BOS detected at index {i}, price: {current_low}")

            # BOS Down: Price breaks below recent structure low
            if len(structure_points_low) >= 2:
                last_low_idx, last_low = structure_points_low[-1]
                prev_low_idx, prev_low = structure_points_low[-2]

                if current_high < prev_low and i > last_low_idx + 1:
                    df.loc[df.index[i], 'bos_down'] = True
                    logger.info(f"Bearish BOS detected at index {i}, price: {current_high}")
                    print(f"Bearish BOS detected at index {i}, price: {current_high}")

        # Identify Change of Character (CHoCH)
        for i in range(window+1, len(df)):
            # Bullish CHoCH: After BOS up, creates higher low
            if df.iloc[i-1]['bos_up']:
                recent_lows = df.iloc[i-window:i]['low'].tolist()
                if min(recent_lows[:-1]) < recent_lows[-1]:
                    df.loc[df.index[i], 'choch_up'] = True
                    logger.info(f"Bullish CHoCH detected at index {i}")
                    print(f"Bullish CHoCH detected at index {i}")

            # Bearish CHoCH: After BOS down, creates lower high
            if df.iloc[i-1]['bos_down']:
                recent_highs = df.iloc[i-window:i]['high'].tolist()
                if max(recent_highs[:-1]) > recent_highs[-1]:
                    df.loc[df.index[i], 'choch_down'] = True
                    logger.info(f"Bearish CHoCH detected at index {i}")
                    print(f"Bearish CHoCH detected at index {i}")

        return df

    def identify_fvg(self):
        """Identify Fair Value Gaps (FVGs)"""
        logger.info("Identifying Fair Value Gaps")
        print("Identifying Fair Value Gaps")
        df = self.df

        # Create separate columns for FVG values to avoid tuple storage issues
        if 'bullish_fvg_low' not in df.columns:
            df['bullish_fvg_low'] = np.nan
            df['bullish_fvg_high'] = np.nan
            df['bullish_fvg_sl_index'] = np.nan

        if 'bearish_fvg_low' not in df.columns:
            df['bearish_fvg_low'] = np.nan
            df['bearish_fvg_high'] = np.nan
            df['bearish_fvg_sl_index'] = np.nan

        # Add mitigation tracking columns
        if 'bullish_fvg_mitigated' not in df.columns:
            df['bullish_fvg_mitigated'] = False

        if 'bearish_fvg_mitigated' not in df.columns:
            df['bearish_fvg_mitigated'] = False

        for i in range(2, len(df)):
            # Bullish FVG: Previous candle's low > Current candle's high
            if df.iloc[i-2]['low'] > df.iloc[i]['high']:
                # Check Gann Box (0-0.5 range for bullish FVGs)
                high_point = df.iloc[i-2]['high']
                low_point = df.iloc[i]['low']
                price_range = high_point - low_point
                fvg_low = df.iloc[i]['high']
                fvg_high = df.iloc[i-2]['low']

                # Calculate relative position in Gann Box (0 to 1)
                if price_range > 0:
                    relative_pos = (fvg_high - low_point) / price_range

                    # Only mark valid FVGs within 0-0.5 Gann range
                    if 0 <= relative_pos <= 0.5:
                        df.loc[df.index[i], 'bullish_fvg'] = True
                        # Store FVG values in separate columns
                        df.loc[df.index[i], 'bullish_fvg_low'] = fvg_low
                        df.loc[df.index[i], 'bullish_fvg_high'] = fvg_high
                        df.loc[df.index[i], 'bullish_fvg_sl_index'] = i

                        logger.info(f"Bullish FVG detected at index {i}, range: {fvg_low}-{fvg_high}")
                        print(f"Bullish FVG detected at index {i}, range: {fvg_low}-{fvg_high}")

            # Bearish FVG: Previous candle's high < Current candle's low
            if df.iloc[i-2]['high'] < df.iloc[i]['low']:
                # Check Gann Box (0.5-1 range for bearish FVGs)
                high_point = df.iloc[i]['high']
                low_point = df.iloc[i-2]['low']
                price_range = high_point - low_point
                fvg_low = df.iloc[i-2]['high']
                fvg_high = df.iloc[i]['low']

                # Calculate relative position in Gann Box (0 to 1)
                if price_range > 0:
                    relative_pos = (fvg_high - low_point) / price_range

                    # Only mark valid FVGs within 0.5-1 Gann range
                    if 0.5 <= relative_pos <= 1:
                        df.loc[df.index[i], 'bearish_fvg'] = True
                        # Store FVG values in separate columns
                        df.loc[df.index[i], 'bearish_fvg_low'] = fvg_low
                        df.loc[df.index[i], 'bearish_fvg_high'] = fvg_high
                        df.loc[df.index[i], 'bearish_fvg_sl_index'] = i

                        logger.info(f"Bearish FVG detected at index {i}, range: {fvg_low}-{fvg_high}")
                        print(f"Bearish FVG detected at index {i}, range: {fvg_low}-{fvg_high}")

        return df

    def check_fvg_mitigation(self, current_idx):
        """Check if any previously identified FVGs have been mitigated"""
        df = self.df

        # Loop through all previous candles
        for i in range(current_idx):
            # Check if the candle had a bullish FVG
            if df.iloc[i].get('bullish_fvg', False) and pd.notna(df.iloc[i].get('bullish_fvg_low')):
                fvg_low = df.iloc[i]['bullish_fvg_low']
                fvg_high = df.iloc[i]['bullish_fvg_high']

                # Check if price revisited the FVG area
                for j in range(i+1, current_idx+1):
                    if df.iloc[j]['low'] <= fvg_high and df.iloc[j]['high'] >= fvg_low:
                        # FVG has been mitigated
                        df.loc[df.index[i], 'bullish_fvg_mitigated'] = True
                        logger.info(f"Bullish FVG at index {i} has been mitigated at index {j}")
                        print(f"Bullish FVG at index {i} has been mitigated at index {j}")
                        break

            # Check if the candle had a bearish FVG
            if df.iloc[i].get('bearish_fvg', False) and pd.notna(df.iloc[i].get('bearish_fvg_low')):
                fvg_low = df.iloc[i]['bearish_fvg_low']
                fvg_high = df.iloc[i]['bearish_fvg_high']

                # Check if price revisited the FVG area
                for j in range(i+1, current_idx+1):
                    if df.iloc[j]['low'] <= fvg_high and df.iloc[j]['high'] >= fvg_low:
                        # FVG has been mitigated
                        df.loc[df.index[i], 'bearish_fvg_mitigated'] = True
                        logger.info(f"Bearish FVG at index {i} has been mitigated at index {j}")
                        print(f"Bearish FVG at index {i} has been mitigated at index {j}")
                        break

        return df

    def execute_trades(self):
        """Execute trades based on SMC signals"""
        logger.info("Starting trade execution backtesting")
        print("Starting trade execution backtesting")
        df = self.df

        # Iterate through each candle for backtesting
        for i in range(5, len(df)):
            current_price = df.iloc[i]['close']

            # Check if we're in a trade
            if self.in_trade:
                # Check if stop loss hit
                if (self.trade_type == 'long' and df.iloc[i]['low'] <= self.stop_loss) or \
                   (self.trade_type == 'short' and df.iloc[i]['high'] >= self.stop_loss):
                    # Stop loss hit
                    self.exit_trade(i, self.stop_loss, 'stop_loss')

                # Check if take profit hit
                elif (self.trade_type == 'long' and df.iloc[i]['high'] >= self.take_profit) or \
                     (self.trade_type == 'short' and df.iloc[i]['low'] <= self.take_profit):
                    # Take profit hit
                    self.exit_trade(i, self.take_profit, 'take_profit')

            else:
                # Update FVG mitigation status
                self.check_fvg_mitigation(i)

                # Check for new trade setups

                # Bullish setup: BOS up + CHoCH up + unmitigated bullish FVG
                if df.iloc[i-1]['bos_up'] and df.iloc[i]['choch_up']:
                    # Look back for unmitigated bullish FVGs
                    for j in range(i-10, i):
                        if j >= 0 and df.iloc[j].get('bullish_fvg', False) and not df.iloc[j].get('bullish_fvg_mitigated', False):
                            fvg_low = df.iloc[j]['bullish_fvg_low']
                            fvg_high = df.iloc[j]['bullish_fvg_high']
                            sl_idx = int(df.iloc[j]['bullish_fvg_sl_index'])

                            # Check if price is near the FVG
                            if fvg_low <= current_price <= fvg_high:
                                # Setup stop loss at the low of the FVG-forming candle
                                stop_loss = df.iloc[sl_idx]['low']

                                # Find recent structure low for take profit
                                recent_lows = df.iloc[i-20:i]['low'].tolist()
                                min_idx = recent_lows.index(min(recent_lows))
                                take_profit = df.iloc[i-20+min_idx]['low']

                                # Enter long trade
                                self.enter_trade(i, current_price, stop_loss, take_profit, 'long')
                                break

                # Bearish setup: BOS down + CHoCH down + unmitigated bearish FVG
                if df.iloc[i-1]['bos_down'] and df.iloc[i]['choch_down']:
                    # Look back for unmitigated bearish FVGs
                    for j in range(i-10, i):
                        if j >= 0 and df.iloc[j].get('bearish_fvg', False) and not df.iloc[j].get('bearish_fvg_mitigated', False):
                            fvg_low = df.iloc[j]['bearish_fvg_low']
                            fvg_high = df.iloc[j]['bearish_fvg_high']
                            sl_idx = int(df.iloc[j]['bearish_fvg_sl_index'])

                            # Check if price is near the FVG
                            if fvg_low <= current_price <= fvg_high:
                                # Setup stop loss at the high of the FVG-forming candle
                                stop_loss = df.iloc[sl_idx]['high']

                                # Find recent structure high for take profit
                                recent_highs = df.iloc[i-20:i]['high'].tolist()
                                max_idx = recent_highs.index(max(recent_highs))
                                take_profit = df.iloc[i-20+max_idx]['high']

                                # Enter short trade
                                self.enter_trade(i, current_price, stop_loss, take_profit, 'short')
                                break

        # Close any open trade at the end of testing
        if self.in_trade:
            self.exit_trade(len(df) - 1, df.iloc[-1]['close'], 'end_of_test')

        logger.info(f"Trade execution completed with {len(self.trades)} trades")
        print(f"Trade execution completed with {len(self.trades)} trades")

        return self.trades, self.equity_curve

    def enter_trade(self, index, price, stop_loss, take_profit, trade_type):
        """Enter a new trade with defined parameters"""
        # Calculate position size based on risk
        risk_amount = self.balance * self.risk_per_trade
        trade_risk = abs(price - stop_loss)
        self.position_size = risk_amount / trade_risk

        # Record trade details
        self.in_trade = True
        self.entry_price = price
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        self.trade_type = trade_type
        self.entry_index = index

        trade_info = {
            'entry_date': self.df.index[index],
            'entry_index': index,
            'entry_price': price,
            'stop_loss': stop_loss,
            'take_profit': take_profit,
            'position_size': self.position_size,
            'risk_amount': risk_amount,
            'type': trade_type,
            'balance_before': self.balance
        }

        logger.info(f"Entered {trade_type} trade at price {price}, SL: {stop_loss}, TP: {take_profit}, Risk: {risk_amount:.2f}")
        print(f"Entered {trade_type} trade at price {price}, SL: {stop_loss}, TP: {take_profit}, Risk: {risk_amount:.2f}")

        return trade_info

    def exit_trade(self, index, price, exit_reason):
        """Exit current trade and update performance metrics"""
        # Calculate profit/loss
        if self.trade_type == 'long':
            pnl = (price - self.entry_price) * self.position_size
        else:  # short
            pnl = (self.entry_price - price) * self.position_size

        # Update balance
        self.balance += pnl
        self.equity_curve.append(self.balance)

        # Record trade outcome
        trade = {
            'entry_date': self.df.index[self.entry_index],
            'entry_index': self.entry_index,
            'entry_price': self.entry_price,
            'exit_date': self.df.index[index],
            'exit_index': index,
            'exit_price': price,
            'stop_loss': self.stop_loss,
            'take_profit': self.take_profit,
            'position_size': self.position_size,
            'pnl': pnl,
            'exit_reason': exit_reason,
            'type': self.trade_type,
            'balance_after': self.balance
        }

        self.trades.append(trade)

        # Reset trade variables
        self.in_trade = False
        self.entry_price = 0
        self.stop_loss = 0
        self.take_profit = 0
        self.position_size = 0
        self.trade_type = None

        logger.info(f"Exited trade at price {price}, Reason: {exit_reason}, P/L: {pnl:.2f}, New Balance: {self.balance:.2f}")
        print(f"Exited trade at price {price}, Reason: {exit_reason}, P/L: {pnl:.2f}, New Balance: {self.balance:.2f}")

        return trade

    def calculate_performance(self):
        """Calculate and return performance metrics"""
        if not self.trades:
            logger.warning("No trades to calculate performance metrics")
            print("No trades to calculate performance metrics")
            return {
                'total_trades': 0,
                'win_rate': 0,
                'profit_factor': 0,
                'total_return_pct': 0,
                'max_drawdown_pct': 0
            }

        # Calculate win rate
        winning_trades = [t for t in self.trades if t['pnl'] > 0]
        win_rate = len(winning_trades) / len(self.trades)

        # Calculate profit factor
        gross_profit = sum([t['pnl'] for t in self.trades if t['pnl'] > 0])
        gross_loss = abs(sum([t['pnl'] for t in self.trades if t['pnl'] <= 0]))
        profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')

        # Calculate total return
        total_return = (self.balance - self.initial_balance) / self.initial_balance

        # Calculate maximum drawdown
        peak = self.initial_balance
        max_drawdown = 0

        for balance in self.equity_curve:
            if balance > peak:
                peak = balance
            drawdown = (peak - balance) / peak
            max_drawdown = max(max_drawdown, drawdown)

        performance = {
            'total_trades': len(self.trades),
            'winning_trades': len(winning_trades),
            'losing_trades': len(self.trades) - len(winning_trades),
            'win_rate': win_rate,
            'profit_factor': profit_factor,
            'total_return': self.balance - self.initial_balance,
            'total_return_pct': total_return * 100,
            'max_drawdown_pct': max_drawdown * 100,
            'final_balance': self.balance
        }

        logger.info(f"Performance metrics: Win rate: {win_rate:.2%}, Profit factor: {profit_factor:.2f}, " +
                     f"Return: {total_return:.2%}, Max drawdown: {max_drawdown:.2%}")
        print(f"Performance metrics: Win rate: {win_rate:.2%}, Profit factor: {profit_factor:.2f}, " +
              f"Return: {total_return:.2%}, Max drawdown: {max_drawdown:.2%}")

        return performance

    def visualize_results(self, start_idx=0, end_idx=None):
        """Visualize backtesting results with trades and SMC patterns"""
        if end_idx is None:
            end_idx = len(self.df)

        # Create figure with subplots
        fig, ax = plt.subplots(figsize=(15, 8))

        # Plot price data
        subset = self.df.iloc[start_idx:end_idx]
        ax.plot(subset.index, subset['close'], label='Close Price', color='black', linewidth=1)

        # Plot FVGs
        for i in range(start_idx, min(end_idx, len(self.df))):
            if self.df.iloc[i].get('bullish_fvg', False) and pd.notna(self.df.iloc[i].get('bullish_fvg_low')):
                fvg_low = self.df.iloc[i]['bullish_fvg_low']
                fvg_high = self.df.iloc[i]['bullish_fvg_high']
                mitigated = self.df.iloc[i].get('bullish_fvg_mitigated', False)
                color = 'lightgreen' if not mitigated else 'darkgreen'
                rect = patches.Rectangle((i-0.5, fvg_low), 1, fvg_high-fvg_low, linewidth=1,
                                        edgecolor=color, facecolor=color, alpha=0.3)
                ax.add_patch(rect)

            if self.df.iloc[i].get('bearish_fvg', False) and pd.notna(self.df.iloc[i].get('bearish_fvg_low')):
                fvg_low = self.df.iloc[i]['bearish_fvg_low']
                fvg_high = self.df.iloc[i]['bearish_fvg_high']
                mitigated = self.df.iloc[i].get('bearish_fvg_mitigated', False)
                color = 'lightcoral' if not mitigated else 'darkred'
                rect = patches.Rectangle((i-0.5, fvg_low), 1, fvg_high-fvg_low, linewidth=1,
                                        edgecolor=color, facecolor=color, alpha=0.3)
                ax.add_patch(rect)

        # Plot BOS and CHoCH
        bos_up_idx = subset[subset['bos_up'] == True].index
        bos_down_idx = subset[subset['bos_down'] == True].index
        choch_up_idx = subset[subset['choch_up'] == True].index
        choch_down_idx = subset[subset['choch_down'] == True].index

        ax.scatter(bos_up_idx, subset.loc[bos_up_idx, 'low'], color='green', marker='^', s=100, label='BOS Up')
        ax.scatter(bos_down_idx, subset.loc[bos_down_idx, 'high'], color='red', marker='v', s=100, label='BOS Down')
        ax.scatter(choch_up_idx, subset.loc[choch_up_idx, 'low'], color='blue', marker='^', s=80, label='CHoCH Up')
        ax.scatter(choch_down_idx, subset.loc[choch_down_idx, 'high'], color='purple', marker='v', s=80, label='CHoCH Down')

        # Plot trades
        for trade in self.trades:
            if start_idx <= trade['entry_index'] < end_idx:
                # Entry point
                color = 'green' if trade['type'] == 'long' else 'red'
                marker = '^' if trade['type'] == 'long' else 'v'
                ax.scatter(trade['entry_index'], trade['entry_price'], color=color, marker=marker, s=120, zorder=5)

                # Exit point
                if trade['exit_index'] < end_idx:
                    color = 'green' if trade['pnl'] > 0 else 'red'
                    ax.scatter(trade['exit_index'], trade['exit_price'], color=color, marker='o', s=120, zorder=5)

                    # Connect entry and exit
                    ax.plot([trade['entry_index'], trade['exit_index']],
                           [trade['entry_price'], trade['exit_price']],
                           color=color, linewidth=1, linestyle='--')

                    # Annotate PnL
                    ax.annotate(f"{trade['pnl']:.2f}",
                              (trade['exit_index'], trade['exit_price']),
                              textcoords="offset points",
                              xytext=(0,10),
                              ha='center')

        ax.set_title('SMC Backtest Results')
        ax.legend(loc='best')
        ax.grid(True, alpha=0.3)

        # Plot equity curve
        fig2, ax2 = plt.subplots(figsize=(15, 5))
        ax2.plot(self.equity_curve, label='Account Balance', color='blue')
        ax2.set_title('Equity Curve')
        ax2.grid(True, alpha=0.3)
        ax2.legend()

        plt.tight_layout()
        return fig, fig2

    def run_backtest(self):
        """Run the full backtest process"""
        logger.info("Starting SMC backtest")
        print("Starting SMC backtest")

        # Identify market structure
        self.identify_structure()

        # Identify Fair Value Gaps
        self.identify_fvg()

        # Execute trades based on signals
        self.execute_trades()

        # Calculate performance metrics
        performance = self.calculate_performance()

        logger.info("Backtest completed successfully")
        print("Backtest completed successfully")

        return {
            'trades': self.trades,
            'performance': performance,
            'equity_curve': self.equity_curve,
            'processed_data': self.df
        }

In [6]:
# Use yfinance to get real market data
def run_smc_backtest(ticker="SPY", start_date="2023-01-01", end_date="2023-12-31", initial_balance=10000, risk_per_trade=0.02):
    """
    Run SMC backtesting on real market data from yfinance

    Parameters:
    ticker (str): Stock ticker symbol
    start_date (str): Start date for historical data in YYYY-MM-DD format
    end_date (str): End date for historical data in YYYY-MM-DD format
    initial_balance (float): Starting account balance
    risk_per_trade (float): Risk percentage per trade (0.02 = 2%)
    """
    print(f"Fetching {ticker} market data from {start_date} to {end_date}")

    # Get historical data from yfinance
    data = yf.download(ticker, start=start_date, end=end_date)

    if data.empty:
        print(f"No data found for {ticker} in the specified date range")
        return None

    print(f"Downloaded {len(data)} days of market data for {ticker}")

    # Initialize and run backtester
    smc = SMCStrategy(data, initial_balance=initial_balance, risk_per_trade=risk_per_trade)
    results = smc.run_backtest()

    # Print performance summary
    print("\n--- PERFORMANCE SUMMARY ---")
    print(f"Total Trades: {results['performance']['total_trades']}")
    print(f"Winning Trades: {results['performance']['winning_trades']}")
    print(f"Win Rate: {results['performance']['win_rate']:.2%}")
    print(f"Profit Factor: {results['performance']['profit_factor']:.2f}")
    print(f"Total Return: ${results['performance']['total_return']:.2f} ({results['performance']['total_return_pct']:.2f}%)")
    print(f"Max Drawdown: {results['performance']['max_drawdown_pct']:.2f}%")
    print(f"Final Balance: ${results['performance']['final_balance']:.2f}")

    # Visualize results
    smc.visualize_results()
    plt.show()

    return results, smc

#results, smc = run_smc_backtest()

In [7]:
class BitMEXLiveTrader:
    def __init__(self, api_key="", api_secret="", test=True, symbol="XBTUSD",
                 timeframe="1h", risk_per_trade=0.02, lookback_periods=100):
        """
        Initialize the BitMEXLiveTrader class

        Parameters:
        api_key (str): BitMEX API key
        api_secret (str): BitMEX API secret
        test (bool): Whether to use testnet (True) or live (False)
        symbol (str): Trading symbol (default XBTUSD)
        timeframe (str): Candle timeframe (1m, 5m, 1h, etc.)
        risk_per_trade (float): Risk percentage per trade (0.02 = 2%)
        lookback_periods (int): Number of candles to fetch for analysis
        """
        self.api_key = api_key
        self.api_secret = api_secret
        self.test = test
        self.symbol = symbol
        self.timeframe = timeframe
        self.risk_per_trade = risk_per_trade
        self.lookback_periods = lookback_periods

        # Initialize BitMEX API client
        self.api = BitMEXTestAPI(
            api_key=self.api_key,
            api_secret=self.api_secret,
            test=self.test
        )

        # Trading state
        self.in_trade = False
        self.trade_type = None
        self.entry_price = None
        self.stop_loss = None
        self.take_profit = None
        self.position_size = 0
        self.trades = []
        self.equity_curve = []
        self.initial_balance = 0
        self.current_balance = 0

        logger.info(f"BitMEXLiveTrader initialized for {symbol} on {timeframe} timeframe")
        print(f"BitMEXLiveTrader initialized for {symbol} on {timeframe} timeframe")

    def get_market_data(self):
        """
        Fetch market data from BitMEX API or fallback to yfinance
        """
        try:
            # Try to get data from BitMEX API
            logger.info(f"Fetching {self.symbol} market data from BitMEX")
            print(f"Fetching {self.symbol} market data from BitMEX")

            # This is a placeholder - you'll need to implement BitMEX data fetching
            # based on your actual API implementation
            data = self.api.get_candles(
                symbol=self.symbol,
                timeframe=self.timeframe,
                count=self.lookback_periods
            )

            # Convert to DataFrame format expected by your SMC logic
            df = pd.DataFrame(data)

            logger.info(f"Retrieved {len(df)} candles from BitMEX")
            print(f"Retrieved {len(df)} candles from BitMEX")

            return df

        except Exception as e:
            # Fallback to yfinance for data (mostly for testing purposes)
            logger.warning(f"Failed to get data from BitMEX API: {str(e)}. Falling back to yfinance.")
            print(f"Failed to get data from BitMEX API: {str(e)}. Falling back to yfinance.")

            # For crypto we'll use a different ticker format
            crypto_ticker = self.symbol.replace('USD', '-USD')
            end_date = datetime.now()
            start_date = end_date - timedelta(days=30)  # Adjust as needed

            data = yf.download(
                crypto_ticker,
                start=start_date.strftime('%Y-%m-%d'),
                end=end_date.strftime('%Y-%m-%d'),
                interval=self.timeframe
            )

            logger.info(f"Retrieved {len(data)} candles from yfinance")
            print(f"Retrieved {len(data)} candles from yfinance")

            return data

    def analyze_market_structure(self, df):
        """
        Apply SMC analysis to the market data
        """
        logger.info("Analyzing market structure using SMC logic")
        print("Analyzing market structure using SMC logic")

        # Create temporary SMCStrategy instance for analysis
        smc = SMCStrategy(df, initial_balance=10000, risk_per_trade=self.risk_per_trade)

        # Identify market structure
        smc.identify_structure()

        # Identify Fair Value Gaps
        smc.identify_fvg()

        # Return the processed dataframe with SMC signals
        return smc.df

    def check_for_signals(self, df):
        """
        Check for trading signals based on the SMC analysis
        """
        logger.info("Checking for trading signals")
        print("Checking for trading signals")

        # Get the latest candle
        current_candle = df.iloc[-1]
        current_price = current_candle['close']

        signal = None

        # If we're already in a trade, check for exit signals
        if self.in_trade:
            if self.trade_type == 'long':
                # Check if stop loss hit
                if current_candle['low'] <= self.stop_loss:
                    signal = {'action': 'exit', 'reason': 'stop_loss', 'price': self.stop_loss}

                # Check if take profit hit
                elif current_candle['high'] >= self.take_profit:
                    signal = {'action': 'exit', 'reason': 'take_profit', 'price': self.take_profit}

            elif self.trade_type == 'short':
                # Check if stop loss hit
                if current_candle['high'] >= self.stop_loss:
                    signal = {'action': 'exit', 'reason': 'stop_loss', 'price': self.stop_loss}

                # Check if take profit hit
                elif current_candle['low'] <= self.take_profit:
                    signal = {'action': 'exit', 'reason': 'take_profit', 'price': self.take_profit}

        else:
            # Check for entry signals, similar to execute_trades in SMCStrategy
            i = len(df) - 1  # Index of current candle

            # Check for bullish setup
            if df.iloc[i-1].get('bos_up', False) and current_candle.get('choch_up', False):
                # Look back for unmitigated bullish FVGs
                for j in range(i-10, i):
                    if j >= 0 and df.iloc[j].get('bullish_fvg', False) and not df.iloc[j].get('bullish_fvg_mitigated', False):
                        fvg_low = df.iloc[j]['bullish_fvg_low']
                        fvg_high = df.iloc[j]['bullish_fvg_high']
                        sl_idx = int(df.iloc[j]['bullish_fvg_sl_index'])

                        # Check if price is near the FVG
                        if fvg_low <= current_price <= fvg_high:
                            # Setup stop loss at the low of the FVG-forming candle
                            stop_loss = df.iloc[sl_idx]['low']

                            # Find recent structure low for take profit
                            recent_lows = df.iloc[i-20:i]['low'].tolist()
                            min_idx = recent_lows.index(min(recent_lows))
                            take_profit = df.iloc[i-20+min_idx]['low']

                            signal = {
                                'action': 'entry',
                                'side': 'long',
                                'price': current_price,
                                'stop_loss': stop_loss,
                                'take_profit': take_profit
                            }
                            break

            # Check for bearish setup
            if df.iloc[i-1].get('bos_down', False) and current_candle.get('choch_down', False):
                # Look back for unmitigated bearish FVGs
                for j in range(i-10, i):
                    if j >= 0 and df.iloc[j].get('bearish_fvg', False) and not df.iloc[j].get('bearish_fvg_mitigated', False):
                        fvg_low = df.iloc[j]['bearish_fvg_low']
                        fvg_high = df.iloc[j]['bearish_fvg_high']
                        sl_idx = int(df.iloc[j]['bearish_fvg_sl_index'])

                        # Check if price is near the FVG
                        if fvg_low <= current_price <= fvg_high:
                            # Setup stop loss at the high of the FVG-forming candle
                            stop_loss = df.iloc[sl_idx]['high']

                            # Find recent structure high for take profit
                            recent_highs = df.iloc[i-20:i]['high'].tolist()
                            max_idx = recent_highs.index(max(recent_highs))
                            take_profit = df.iloc[i-20+max_idx]['high']

                            signal = {
                                'action': 'entry',
                                'side': 'short',
                                'price': current_price,
                                'stop_loss': stop_loss,
                                'take_profit': take_profit
                            }
                            break

        return signal

    def execute_signal(self, signal):
        """
        Execute the trading signal via BitMEX API
        """
        if signal is None:
            logger.info("No trading signal detected")
            print("No trading signal detected")
            return

        if signal['action'] == 'entry':
            self.execute_entry(signal)
        elif signal['action'] == 'exit':
            self.execute_exit(signal)

    def execute_entry(self, signal):
        """
        Execute an entry order based on the signal
        """
        side = signal['side']
        price = signal['price']
        stop_loss = signal['stop_loss']
        take_profit = signal['take_profit']

        logger.info(f"=== OPENING {side.upper()} POSITION ===")
        print(f"=== OPENING {side.upper()} POSITION ===")

        # Calculate position size based on risk
        account_info = self.api.get_profile_info()
        self.current_balance = account_info['balance']  # Adjust based on your API's response format

        risk_amount = self.current_balance * self.risk_per_trade
        risk_per_unit = abs(price - stop_loss)
        position_size = risk_amount / risk_per_unit

        # Round position size to appropriate precision
        position_size = max(1, round(position_size))  # Minimum size of 1

        logger.info(f"Entry Price: {price}, Stop Loss: {stop_loss}, Take Profit: {take_profit}")
        logger.info(f"Position Size: {position_size} contracts")
        print(f"Entry Price: {price}, Stop Loss: {stop_loss}, Take Profit: {take_profit}")
        print(f"Position Size: {position_size} contracts")

        try:
            # Execute order via BitMEX API
            order_side = "Buy" if side == "long" else "Sell"
            order_result = self.api.open_test_position(side=order_side, quantity=position_size)

            # Update trading state
            self.in_trade = True
            self.trade_type = side
            self.entry_price = price
            self.stop_loss = stop_loss
            self.take_profit = take_profit
            self.position_size = position_size

            logger.info(f"{side.capitalize()} position opened successfully")
            print(f"{side.capitalize()} position opened successfully")

            # Record trade
            trade = {
                'entry_time': datetime.now(),
                'entry_price': price,
                'side': side,
                'position_size': position_size,
                'stop_loss': stop_loss,
                'take_profit': take_profit,
                'risk_amount': risk_amount
            }
            self.trades.append(trade)

        except Exception as e:
            logger.error(f"Error opening {side} position: {str(e)}")
            print(f"Error opening {side} position: {str(e)}")

    def execute_exit(self, signal):
        """
        Execute an exit order based on the signal
        """
        reason = signal['reason']
        price = signal['price']

        logger.info(f"=== CLOSING {self.trade_type.upper()} POSITION ({reason}) ===")
        print(f"=== CLOSING {self.trade_type.upper()} POSITION ({reason}) ===")

        try:
            # Close position via BitMEX API
            self.api.close_all_positions()

            # Calculate profit/loss
            if self.trade_type == 'long':
                pnl = (price - self.entry_price) * self.position_size
            else:  # short
                pnl = (self.entry_price - price) * self.position_size

            # Update the latest trade record
            current_trade = self.trades[-1]
            current_trade['exit_time'] = datetime.now()
            current_trade['exit_price'] = price
            current_trade['exit_reason'] = reason
            current_trade['pnl'] = pnl

            # Update account balance
            self.api.get_profile_info()  # Refresh account info

            logger.info(f"Position closed with P&L: {pnl}")
            print(f"Position closed with P&L: {pnl}")

            # Reset trading state
            self.in_trade = False
            self.trade_type = None
            self.entry_price = None
            self.stop_loss = None
            self.take_profit = None
            self.position_size = 0

        except Exception as e:
            logger.error(f"Error closing position: {str(e)}")
            print(f"Error closing position: {str(e)}")

    def run(self, scan_interval=60):
        """
        Main loop for live trading

        Parameters:
        scan_interval (int): Seconds between market scans
        """
        logger.info("Starting BitMEXLiveTrader")
        print("Starting BitMEXLiveTrader")

        # Display initial profile info
        logger.info("=== INITIAL PROFILE ===")
        print("=== INITIAL PROFILE ===")
        profile = self.api.get_profile_info()
        self.initial_balance = profile['balance']  # Adjust based on your API's response format
        self.current_balance = self.initial_balance

        # Main trading loop
        try:
            while True:
                logger.info(f"Scanning market at {datetime.now()}")
                print(f"Scanning market at {datetime.now()}")

                # Get market data
                df = self.get_market_data()

                # Analyze market structure
                df_analyzed = self.analyze_market_structure(df)

                # Check for signals
                signal = self.check_for_signals(df_analyzed)

                # Execute signal if any
                self.execute_signal(signal)

                # Show current balance after trade execution
                if signal is not None:
                    logger.info("=== CURRENT PROFILE ===")
                    print("=== CURRENT PROFILE ===")
                    profile = self.api.get_profile_info()
                    self.current_balance = profile['balance']  # Adjust based on your API's response format

                # Wait for next scan
                logger.info(f"Waiting for {scan_interval} seconds until next scan...")
                print(f"Waiting for {scan_interval} seconds until next scan...")
                time.sleep(scan_interval)

        except KeyboardInterrupt:
            logger.info("BitMEXLiveTrader stopped by user")
            print("BitMEXLiveTrader stopped by user")

            # Close any open positions
            if self.in_trade:
                logger.info("=== CLOSING OPEN POSITIONS ===")
                print("=== CLOSING OPEN POSITIONS ===")
                self.api.close_all_positions()
                self.in_trade = False

            # Show final account info
            logger.info("=== FINAL PROFILE ===")
            print("=== FINAL PROFILE ===")
            self.api.get_profile_info()

        except Exception as e:
            logger.error(f"BitMEXLiveTrader error: {str(e)}")
            print(f"BitMEXLiveTrader error: {str(e)}")

            # Try to close positions on error
            if self.in_trade:
                try:
                    logger.info("=== CLOSING POSITIONS ON ERROR ===")
                    print("=== CLOSING POSITIONS ON ERROR ===")
                    self.api.close_all_positions()
                except:
                    pass

        finally:
            # Display summary
            logger.info("=== TRADING SUMMARY ===")
            print("=== TRADING SUMMARY ===")
            logger.info(f"Total trades: {len(self.trades)}")
            print(f"Total trades: {len(self.trades)}")

            winning_trades = sum(1 for trade in self.trades if trade.get('pnl', 0) > 0)
            logger.info(f"Winning trades: {winning_trades}")
            print(f"Winning trades: {winning_trades}")

            if len(self.trades) > 0:
                win_rate = winning_trades / len(self.trades)
                logger.info(f"Win rate: {win_rate:.2%}")
                print(f"Win rate: {win_rate:.2%}")

            total_pnl = sum(trade.get('pnl', 0) for trade in self.trades)
            logger.info(f"Total P&L: {total_pnl}")
            print(f"Total P&L: {total_pnl}")

            if self.initial_balance > 0:
                return_pct = total_pnl / self.initial_balance * 100
                logger.info(f"Return: {return_pct:.2f}%")
                print(f"Return: {return_pct:.2f}%")

In [8]:
def mainBitMEXLiveTrader():
    """
    Main function to run the BitMEXLiveTrader
    """
    # BitMEX API credentials (use your test API key and secret)
    API_KEY = ""  # Your test API key
    API_SECRET = ""  # Your test API secret

    try:
        # Initialize and run BitMEXLiveTrader
        trader = BitMEXLiveTrader(
            api_key=API_KEY,
            api_secret=API_SECRET,
            test=True,  # Use testnet
            symbol="XBTUSD",  # Bitcoin/USD
            timeframe="1h",  # 1-hour candles
            risk_per_trade=0.02  # 2% risk per trade
        )

        # Start trading loop
        trader.run(scan_interval=300)  # Scan every 5 minutes

    except KeyboardInterrupt:
        logger.info("BitMEXLiveTrader stopped by user")
        print("BitMEXLiveTrader stopped by user")
    except Exception as e:
        logger.error(f"Unexpected error: {str(e)}")
        print(f"Unexpected error: {str(e)}")

In [9]:
 mainBitMEXLiveTrader()

/usr/local/lib/python3.11/dist-packages/swagger_spec_validator/validator20.py:49: SwaggerValidationWarning: Found "$ref: #/definitions/UserPreferences" with siblings that will be overwritten. See https://stackoverflow.com/a/48114924 for more information. (path #/definitions/User/properties/preferences)
  warnings.warn(


BitMEXTestAPI initialized for XBTUSD on testnet
BitMEXLiveTrader initialized for XBTUSD on 1h timeframe
Starting BitMEXLiveTrader
=== INITIAL PROFILE ===


/usr/local/lib/python3.11/dist-packages/bravado_core/spec.py:469: Warning: JSON format is not registered with bravado-core!
  warnings.warn(


Profile information retrieved successfully
Account: mmehgoss+bitmexdemo@gmail.com
Wallet Balance: 0.01000000 XBT
Available Margin: 0.01000000 XBT
No open positions
Scanning market at 2025-03-23 06:37:20.055335
Fetching XBTUSD market data from BitMEX
Failed to get data from BitMEX API: 'BitMEXTestAPI' object has no attribute 'get_candles'. Falling back to yfinance.
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Retrieved 720 candles from yfinance
Analyzing market structure using SMC logic
Initializing SMC Strategy Backtester with 720 candles
Strategy initialized with 10000 account balance and 2.0% risk per trade
Identifying market structure
Bearish BOS detected at index 113, price: 0.01088982354849577
Bearish CHoCH detected at index 114
Identifying Fair Value Gaps
Bullish FVG detected at index 19, range: 0.010899399407207966-0.010900110937654972
Bearish FVG detected at index 25, range: 0.0108967749401927-0.010897627100348473
Bullish FVG detected at index 93, range: 0.010898159816861153-0.010898551903665066
Bullish FVG detected at index 112, range: 0.010891109704971313-0.010891346260905266
Bullish FVG detected at index 121, range: 0.010886901058256626-0.010887627489864826
Bullish FVG detected at index 200, range: 0.010896239429712296-0.010896694846451283
Bullish FVG detected at index 659, range: 0.010898519307374954-0.01089982595294714
Checking for trading signals
No trading signal detected
Wa

ERROR:__main__:Unexpected error: '>' not supported between instances of 'dict' and 'int'


Profile information retrieved successfully
Account: mmehgoss+bitmexdemo@gmail.com
Wallet Balance: 0.01000000 XBT
Available Margin: 0.01000000 XBT
No open positions
=== TRADING SUMMARY ===
Total trades: 0
Winning trades: 0
Total P&L: 0
Unexpected error: '>' not supported between instances of 'dict' and 'int'
